Install nuget package
- install semantic kernel 1.22.0
- install MistralAI connector 1.22.0-alpha

In [ ]:
#r "nuget: Microsoft.SemanticKernel, 1.22.0"
#r "nuget: Microsoft.SemanticKernel.Connectors.MistralAI, 1.22.0-alpha"
#r "nuget: System.Text.Json, 7.0.0"

The below script needs to be able to find the current output cell; this is an easy method to get it.

import packages

In [2]:

using Microsoft.SemanticKernel;
using Microsoft.SemanticKernel.ChatCompletion;
using Microsoft.SemanticKernel.Connectors.MistralAI;

using System.ComponentModel;
using System;
using System.Collections.Generic;
using System.ComponentModel;
using System.Linq;
using System.Text.Json.Serialization;
using System.Threading.Tasks;


(9,7): warning CS0105: The using directive for 'System.ComponentModel' appeared previously in this namespace



Create Plugin function

In [3]:


public class LightsPlugin
{
   // Mock data for the lights
   private readonly List<LightModel> lights = new()
   {
      new LightModel { Id = 1, Name = "Table Lamp", IsOn = false },
      new LightModel { Id = 2, Name = "Porch light", IsOn = false },
      new LightModel { Id = 3, Name = "Chandelier", IsOn = true }
   };

   [KernelFunction("get_lights")]
   [Description("Gets a list of lights and their current state")]
   [return: Description("An array of lights")]
   public async Task<List<LightModel>> GetLightsAsync()
   {
      return lights;
   }

   [KernelFunction("change_state")]
   [Description("Changes the state of the light")]
   [return: Description("The updated state of the light; will return null if the light does not exist")]
   public async Task<LightModel?> ChangeStateAsync(int id, bool isOn)
   {
      var light = lights.FirstOrDefault(light => light.Id == id);

      if (light == null)
      {
         return null;
      }

      // Update the light with the new state
      light.IsOn = isOn;

      return light;
   }
}

public class LightModel
{
   [JsonPropertyName("id")]
   public int Id { get; set; }

   [JsonPropertyName("name")]
   public string Name { get; set; }

   [JsonPropertyName("is_on")]
   public bool? IsOn { get; set; }
}


(24,32): warning CS8632: The annotation for nullable reference types should only be used in code within a '#nullable' annotations context.

(16,40): warning CS1998: This async method lacks 'await' operators and will run synchronously. Consider using the 'await' operator to await non-blocking API calls, or 'await Task.Run(...)' to do CPU-bound work on a background thread.

(24,35): warning CS1998: This async method lacks 'await' operators and will run synchronously. Consider using the 'await' operator to await non-blocking API calls, or 'await Task.Run(...)' to do CPU-bound work on a background thread.



Create Semantic Kernel builder

In [4]:
// create and build Kernel using Mistral AI configuration
#pragma warning disable SKEXP0070
IKernelBuilder kernelBuilder = Kernel.CreateBuilder();
kernelBuilder.AddMistralChatCompletion(
    modelId: "mistral-small-latest",
    apiKey: "<APIKey>"
);
Kernel kernel = kernelBuilder.Build();
var chatCompletionService = kernel.GetRequiredService<IChatCompletionService>();

Add plugin to Kernel

In [5]:
// Add a plugin (the LightsPlugin class is defined below)
kernel.Plugins.AddFromType<LightsPlugin>("Lights");


Call ChatCompletion service and get response
- Enable Planner
    - Define Execution Setting for Mistral AI (ToolCallBehavior Auto)
- Create chat prompt message (input) and invoke kernel

In [11]:

#pragma warning disable SKEXP0070
// Enable planning with Mistral AI settings
var execution_settings = new MistralAIPromptExecutionSettings
{
    ToolCallBehavior = MistralAIToolCallBehavior.AutoInvokeKernelFunctions
};

//User input
const string chatPrompt = "<message role=\"user\"> Gets a list of lights and their current state</message>";

var chatSemanticFunction = kernel.CreateFunctionFromPrompt(chatPrompt,execution_settings);

// Get the response from the AI
var chatPromptResult = await kernel.InvokeAsync(chatSemanticFunction);
// Print the results
Console.WriteLine("Assistant > " + chatPromptResult);

Assistant > Here are the lights and their current states:

- Table Lamp: Off
- Porch light: Off
- Chandelier: On

Which light would you like to turn on or off?
